In [1]:
import pandas as pd
import data_util
from tec_an import TecAn
import numpy as np
from data_util import *
from bitstamp import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from data_agent import *
from stock_agent import *
from backtest import *

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *


import xgboost as xgb


In [2]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = [],
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = ["btcusd"],
                 val_start = 1234,
                 val_end = 1234,
                 train_start_list = []
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [3]:
win = 40
minus = 15
step =  1

tec = TecAn(windows = win, windows_limit = 100)
sourceDataGenerator = SourceDataGenerator(tec = tec)
online = get_online_data(minus, sourceDataGenerator, True, win)

In [4]:
train_x_raw, train_y_raw = online.load_train_data()
x, y = prepare_train_data(train_x_raw, train_y_raw , step)


In [5]:
past = [ i for i in range(1)]

scoring = [
    #'accuracy', 
    'f1', 
    'precision', 
    'recall'
]

score = 'f1'

In [6]:

def getModel():
    param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
    #param = {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 3, 'random_state': 42}
    coModel = RandomForestClassifier(**param)
    return coModel

def rand():
    ##print("rand created")
    param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
    return RandomForestClassifier(**param)


In [7]:
from sklearn.feature_selection import SelectFromModel
from sklearn.gaussian_process import GaussianProcessClassifier
from catboost import CatBoostClassifier
from sklearn.feature_selection import RFE, RFECV
from pytorch_tabnet.tab_model import TabNetClassifier

def test(score, n_in):
    
    train_x_raw, train_y_raw = online.load_train_data()
    train_x, train_y, _closed_prices = get_sequencial_data(train_x_raw, train_y_raw , step)
    
    train_x, train_y = get_under_balanced_set(train_x, train_y)
    #train_x, train_y = get_balanced_set(train_x, train_y)

    val_x_raw, val_y_raw = online.load_val_data("btcusd")
    val_x, val_y, closed_prices = get_sequencial_data(val_x_raw, val_y_raw , step)


    svc = DecisionTreeClassifier(random_state=10)
    #svc = RandomForestClassifier(random_state=10)
    #svc = xgb.XGBClassifier(random_state=10)
    
    
    cls = xgb.XGBClassifier(random_state=1,
                                       objective = "binary:logistic",
                                       eval_metric='mlogloss',
                                       learning_rate=0.01, 
                                       #use_label_encoder = True
                           )
    
    
    
    cls = CatBoostClassifier(logging_level = 'Silent')
    
    cls = make_pipeline(StandardScaler(),
                                MLPClassifier(
                                    solver='lbfgs', alpha=1, random_state=1, max_iter=20000,
                                    early_stopping=True)
                                )
    

    cls = DecisionTreeClassifier(random_state=10)

    clf = TabNetClassifier(verbose=0)   

    cls = rand()
    
    rfecv = RFECV(estimator=svc, scoring=score)
    #rfecv = RFECV(estimator=svc)
    #rfecv = SelectFromModel(estimator=svc)
    #rfecv = RFE(estimator=RandomForestClassifier(random_state=10), n_features_to_select=10)

    pipe = make_pipeline(Normalizer(),
                         rfecv,
                         cls)
    
    #train_by_step(pipe, step, online)
    #print(Counter(train_y))
    pipe.fit(train_x, train_y)
    
    back, metrics = eval_step(pipe, "btcusd", step, online, verbose = False)

    print(f"{score} - {back.current} | {metrics}")
    
    return rfecv

In [8]:
#test("precision", 2)

In [9]:
for p in past:
    print(f"Past {p}")
    #test(score, p)
    for score in scoring:
        rfecv = test(score, p)
        #break
        

Past 0


KeyboardInterrupt: 

In [ ]:
preds = pipe.predict(xx)

In [ ]:
back = BackTest(value = 100, 
                verbose = True, 
                sell_on_profit = True,
                pending_sell_steps = step)


#backtest_baseline(x, y, closed_prices, step, back)
backtest_baseline(x, preds, closed_prices, step, back)

print(back.current)

In [ ]:
print(rfecv)
min_features_to_select = rfecv.min_features_to_select
print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(min_features_to_select,
               len(rfecv.grid_scores_) + min_features_to_select),
         rfecv.grid_scores_)
plt.show()

In [ ]:
pd.DataFrame(rfecv.support_)

In [ ]:
 datetime.datetime.fromtimestamp(1626174900)

In [ ]:
def load_online(minutes, window, val_end, currency = "btcusd"):
    tec = TecAn(windows = window, windows_limit = 100)
    source_data_generator = SourceDataGenerator(tec = tec)


    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = [],
                 train_limit = 40,
                 val_limit = 1000,
                 val_keys = [currency],
                 val_start = val_end - (60),
                 val_end = val_end,
                 train_start_list = []
    )

    start = val_end - (60 * 100 * minutes)
    end = val_end - (60 * minutes)

    online.load_val_cache(
                    val_keys = [currency],                  
                    start = start,
                    end = end)
    return online

In [ ]:
load_online(15, 30, 1627217023)

In [ ]:
def order_by_proft(e):
    return e['profit']
    #return e['models_profit_metric']['btcusd']

In [ ]:
results = load("model/decision")

In [ ]:
results.sort(key=order_by_proft, reverse = False)

In [ ]:
results[-1]